# HPC with TensorFlow 2

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import zfit
from zfit import z
import numpy as np
import numba

ModuleNotFoundError: No module named 'numba'

As mentioned, TensorFlow is basically Numpy. Let's check that out

In [ ]:
rnd1 = tf.random.uniform(shape=(10,),  # notice the "shape" argument: it's more picky than Numpy
                         minval=0,
                         maxval=10)
rnd2 = tf.random.uniform(shape=(10,),
                         maxval=10)

In [ ]:
rnd1

This is in fact a "numpy array wrapped" and can explicitly be converted to an array

In [ ]:
rnd1.numpy()

Other operations act as we would expect it

In [ ]:
rnd1 + 10

... and it converts itself (often) to Numpy when needed.

In [ ]:
np.sqrt(rnd1)

We can slice it...

In [ ]:
rnd1[1:3]

...expand it....

In [ ]:
rnd1[None, :, None]

...and broadcast with the known (maybe slightly stricter) rules

In [ ]:
matrix1 = rnd1[None, :] * rnd1[:, None]

## Equivalent operations

Many operations that exist in Numpy also exist in TensorFlow, sometimes with a different name.

In [ ]:
tf.sqrt(rnd1)

In [ ]:
tf.reduce_sum(matrix1, axis=0)  # with the axis argument to specify over which to reduce

## TensorFlow kernels

In general, TensorFlow is preciser compared to Numpy and does less automatic dtype casting and asks more explicit for shapes. For example, integers don't work in the logarithm. However, this error message illustrates very well the kernel dispatch system of TensorFlow.

In [ ]:
try:
    tf.math.log(5)
except tf.errors.NotFoundError as error:
    print(error)

What we see here: it searches the registered kernels and does not find any that supports this operation. We find different classifications:
- GPU: normal GPU kernel
- CPU: normal CPU kernel
- XLA: [Accelerated Linear Algebra](https://www.tensorflow.org/xla) is a high-level compiler that can fuse operations, which would result in single calls to a kernel, to a single kernel.

## tf.function

We now want to see the JIT in action. Therefore, we use the example from the slides and start modifying it.

In [ ]:
def add_log(x, y):
    print('running Python')
    tf.print("running TensorFlow")
    x_sq = tf.math.log(x)
    y_sq = tf.math.log(y)
    return x_sq + y_sq

As seen before, we can use it like Python. To make sure that we know when the actual Python is executed, we inserted a print and a `tf.print`, the latter is a TensorFlow operation and therefore expected to be called everytime we compute something.

In [ ]:
add_log(4., 5.)

In [ ]:
add_log(42., 52.)

As we see, both the Python and TensorFlow operation execute. Now we can do the same with a decorator. Note that so far we entered pure Python numbers, not Tensors. Since we ran in eager mode, this did not matter so far.

In [ ]:
@tf.function(autograph=False)
def add_log_tf(x, y):
    print('running Python')
    tf.print("running TensorFlow")
    x_sq = tf.math.log(x)
    y_sq = tf.math.log(y)
    return x_sq + y_sq

In [ ]:
add_log_tf(1., 2.)

In [ ]:
add_log_tf(11., 21.)  # again with different numbers

As we see, Python is still run: this happens because 11. is not equal to 1., TensorFlow does not convert those to Tensors. Lets use it in the right way, with Tensors

In [ ]:
add_log_tf(tf.constant(1.), tf.constant(2.))  # first compilation

In [ ]:
add_log_tf(tf.constant(11.), tf.constant(22.))

Now only the TensorFlow operations get executed! Everything else became static. We can illustrate this more extremely here

In [ ]:
@tf.function(autograph=True)
def add_rnd(x):
    print('running Python')
    tf.print("running TensorFlow")
    rnd_np = np.random.uniform()
    rnd_tf = tf.random.uniform(shape=())
    return x * rnd_np, x * rnd_tf

In [ ]:
add_rnd(tf.constant(1.))

The first time, the numpy code was executed as well, no difference so far. However, running it a second time, only the TensorFlow parts can change

In [ ]:
add_rnd(tf.constant(1.))

In [ ]:
add_rnd(tf.constant(2.))

We see now clearly: TensorFlow executes the function but _only cares about the TensorFlow operations_, everything else is regarded as static. This can be a large pitfall! If we would execute this function _without_ the decorator, we would get a different result, since Numpy is also sampling a new random variable every time

## Variables

TensorFlow offers the possibility to have statefull objects inside a compiled graph (which e.g. is not possible with Numba). The most commonly used one is the `tf.Variable`. Technically, they are automatically captured on the function compilation and belong to it.

In [ ]:
var1 = tf.Variable(1.)

In [ ]:
@tf.function(autograph=True)
def scale_by_var(x):
    print('running Python')
    tf.print("running TensorFlow")
    return x * var1

In [ ]:
scale_by_var(tf.constant(1.))

In [ ]:
scale_by_var(tf.constant(2.))

In [ ]:
var1.assign(42.)
scale_by_var(tf.constant(1.))

As we see, the output changed. This is of course especially useful in the context of model fitting libraries, be it likelihoods or neural networks.

In [ ]:
def add_rnd(x):
    print('running Python')
    tf.print("running TensorFlow")
    rnd_np = np.random.uniform()
    rnd_tf = tf.random.uniform(shape=())
    return x * rnd_np, x * rnd_tf

In [ ]:
add_rnd(tf.constant(1.))

In [ ]:
add_rnd(tf.constant(2.))

This means that we can use Numpy fully compatible in eager mode, but not when decorated.

In [ ]:
def try_np_sqrt(x):
    return np.sqrt(x)

In [ ]:
try_np_sqrt(tf.constant(5.))

In [ ]:
try_np_sqrt_tf = tf.function(try_np_sqrt, autograph=False)  # equivalent to decorator

In [ ]:
try:
    try_np_sqrt_tf(tf.constant(5.))
except NotImplementedError as error:
    print(error)

As we see, Numpy complains in the graph mode, given that it cannot handle the Symbolic Tensor.

Having the `tf.function` decorator means that we can't use any Python dynamicity. What fails when decorated but works nicely if not:

In [ ]:
def greater_python(x, y):
    if x > y:
        return True
    else:
        return False

In [ ]:
greater_python(tf.constant(1.), tf.constant(2.))

This works again, and will fail with the graph decorator.

In [ ]:
greater_python_tf = tf.function(greater_python, autograph=False)

In [ ]:
try:
    greater_python_tf(tf.constant(1.), tf.constant(2.))
except Exception as error:
    print(error)

The error message hints at something: while this does not work now - Python does not yet now the value of the Tensors so it can't decide whether it will evaluate to True or False - there is the possibility of "autograph": it automatically converts (a subset) of Python to TensorFlow: while loops, for loops through Tensors and conditionals. However, this is usually less effective and more errorprone than using explicitly the `tf.*` functions. Lets try it!

In [ ]:
greater_python_tf_autograph = tf.function(greater_python, autograph=True)

In [ ]:
greater_python_tf_autograph(tf.constant(1.), tf.constant(2.))

This now works neatless! But we're never sure.

## Performance

In the end, this is what matters. And a comparison would be nice. Let's do that and see how Numpy and TensorFlow compare.

In [ ]:
nevents = 10000000
data_tf = tf.random.uniform(shape=(nevents,), dtype=tf.float64)
data_np = np.random.uniform(size=(nevents,))

In [ ]:
def calc_np(x):
    x_init = x
    i = 42.
    x = np.sqrt(np.abs(x_init * (i + 1.)))
    x = np.cos(x - 0.3)
    x = np.power(x, i + 1)
    x = np.sinh(x + 0.4)
    x = x ** 2
    x = x / np.mean(x)
    x = np.abs(x)
    logx = np.log(x)
    x = np.mean(logx)
    
    x1 = np.sqrt(np.abs(x_init * (i + 1.)))
    x1 = np.cos(x1 - 0.3)
    x1 = np.power(x1, i + 1)
    x1 = np.sinh(x1 + 0.4)
    x1 = x1 ** 2
    x1 = x1 / np.mean(x1)
    x1 = np.abs(x1)
    logx = np.log(x1)
    x1 = np.mean(logx)
    
    x2 = np.sqrt(np.abs(x_init * (i + 1.)))
    x2 = np.cos(x2 - 0.3)
    x2 = np.power(x2, i + 1)
    x2 = np.sinh(x2 + 0.4)
    x2 = x2 ** 2
    x2 = x2 / np.mean(x2)
    x2 = np.abs(x2)
    logx = np.log(x2)
    x2 = np.mean(logx)
    return x + x1 + x2

calc_np_numba = numba.jit(nopython=True, parallel=True)(calc_np)

In [ ]:
def calc_tf(x):
    x_init = x
    i = 42.
    x = tf.sqrt(tf.abs(x_init * (tf.cast(i, dtype=tf.float64) + 1.)))
    x = tf.cos(x - 0.3)
    x = tf.pow(x, tf.cast(i + 1, tf.float64))
    x = tf.sinh(x + 0.4)
    x = x ** 2
    x = x / tf.reduce_mean(x)
    x = tf.abs(x)
    x = tf.reduce_mean(tf.math.log(x))
    
    x1 = tf.sqrt(tf.abs(x_init * (tf.cast(i, dtype=tf.float64) + 1.)))
    x1 = tf.cos(x1 - 0.3)
    x1 = tf.pow(x1, tf.cast(i + 1, tf.float64))
    x1 = tf.sinh(x1 + 0.4)
    x1 = x1 ** 2
    x1 = x1 / tf.reduce_mean(x1)
    x1 = tf.abs(x1)
    
    x2 = tf.sqrt(tf.abs(x_init * (tf.cast(i, dtype=tf.float64) + 1.)))
    x2 = tf.cos(x2 - 0.3)
    x2 = tf.pow(x2, tf.cast(i + 1, tf.float64))
    x2 = tf.sinh(x2 + 0.4)
    x2 = x2 ** 2
    x2 = x2 / tf.reduce_mean(x2)
    x2 = tf.abs(x2)
    
    return x + x1 + x2

calc_tf_func = tf.function(calc_tf, autograph=False)

In [ ]:
%%timeit -n1 -r1  # compile time, just for curiosity
calc_tf_func(data_tf)

In [ ]:
%%timeit -n1 -r1  # compile time, just for curiosity
calc_np_numba(data_np)

In [ ]:
%timeit calc_np(data_np)  # not compiled

In [ ]:
%timeit calc_tf(data_tf)  # not compiled

In [ ]:
%%timeit -n1 -r7
calc_np_numba(data_np)

In [ ]:
%%timeit -n1 -r7
calc_tf_func(data_tf)

We can now play around with this numbers. Depending on the size (we can go up to 10 mio) and parallelizability of the problem, the numbers differ..

In general:
- Numpy is faster for small numbers
- TensorFlow is faster for larger arrays and well parallelizable computations. Due to the larger overhead in dispatching in eager mode, it is significantly slower for very small (1-10) sample sizes.

=> there is no free lunch

Note: this has not run on a GPU, which would automatically happen for TensorFlow.

## Control flow

While TensorFlow is independent of the Python control flow, it has its own functions for that, mainly:
- while_loop(): a while loop taking a body and condition function
- cond(): if-like
- case and switch_case: if/elif statements
- tf.where

In [ ]:
def true_fn():
    return 1.

def false_fn():
    return 0.

value = tf.cond(tf.greater(111., 42.), true_fn=true_fn, false_fn=false_fn)

In [ ]:
value

## Gradients

TensorFlow allows us to calculate the automatic gradients.

In [ ]:
var2 = tf.Variable(2.)

In [ ]:
with tf.GradientTape() as tape:
    tape.watch(var2)  # actually watches all variables already by default
    y = var2 ** 3
y

In [ ]:
grad = tape.gradient(y, var2)
grad

This allows to do many things with gradients and e.g. solve differential equations.

## Statistics

While TensorFlow offers some support for statistical inference, TensorFlow-Probability is very strong at this and provides MCMC methods, probability distributions and more.

In [ ]:
cauchy = tfp.distributions.Cauchy(loc=1., scale=10.)

In [ ]:
sample = cauchy.sample(10)

In [ ]:
cauchy.prob(sample)

### How TFP compares to zfit

TensorFlow-Probability offers a great choice of distributions to build a model. The flexibility in terms of vectorization and parametrization is larger than in zfit. However, they only provide analytic models and lack any numerical normalization or samplings.

Internally, zfit simply wraps the for certain implementations. There is also a standard wrapper, `WrapDistribution`, that allows to easily wrap any TFP distribution and use it in zfit.

# HowTo with zfit

Whenever possible, it is preferrable to write anything in TensorFlow. But given the possibility to mix, we can use this.
- try to use `z.py_function` or `tf.py_function` to wrap pure Python code
- if you write something and want to make sure it is run in eager mode, use `zfit.run.assert_executing_eagerly()`. This way, your function won't be compiled and an error would be raised.
- set the graph mode and numerical gradient accordingly

In [ ]:
x_tf = z.constant(42.)

def sqrt(x):
    return np.sqrt(x)

y = z.py_function(func=sqrt, inp=[x_tf], Tout=tf.float64)

In [ ]:
zfit.run.set_graph_mode(False)
zfit.run.set_autograd_mode(False)

In [ ]:
class NumpyGauss(zfit.pdf.ZPDF):
    _PARAMS = ['mu', 'sigma']
    
    def _unnormalized_pdf(self, x):
        zfit.run.assert_executing_eagerly()  # make sure we're eager
        data = z.unstack_x(x)
        mu = self.params['mu']
        sigma = self.params['sigma']
        return tf.convert_to_tensor(np.exp( - 0.5 * (data - mu) ** 2 / sigma ** 2))

In [ ]:
obs = zfit.Space('obs1', (-3, 3))
mu = zfit.Parameter('mu', 0., -1, 1)
sigma = zfit.Parameter('sigma', 1., 0.1, 10)


In [ ]:
gauss_np = NumpyGauss(obs=obs, mu=mu, sigma=sigma)
gauss = zfit.pdf.Gauss(obs=obs, mu=mu, sigma=sigma)

In [ ]:
integral_np = gauss_np.integrate((-1, 0))
integral = gauss.integrate((-1, 0))
print(integral_np, integral)

### What is 'z'?

This is a subset of TensorFlow, wrapped to improve dtype handling and sometimes even provide additional functionality, such as `z.function` decorator.